In [1]:
import pandas as pd
import os
import json

In [2]:
df = pd.read_csv(r'C:\Users\rocci\OneDrive\Desktop\Università\TesiMagistrale\dataPreparationTool\backend\dataset.csv')

In [7]:
def exec_operation(operation, df4):
   df4 = eval('df4.' + operation)
   print(df4.head())
   return df4



def exec_pipeline(operations):
   df = pd.read_csv(r'C:\Users\rocci\OneDrive\Desktop\Università\TesiMagistrale\dataPreparationTool\backend\dataset.csv')
   for operation in operations:
      df = exec_operation(operation, df)
   return df

In [8]:
lista = ['select_dtypes(include="float64")', 'add(1)']
df2 = exec_pipeline(lista)

   Score
0   75.5
1   71.6
2   67.8
3   60.5
4   32.1
   Score
0   76.5
1   72.6
2   68.8
3   61.5
4   33.1


In [120]:
function = 'df.' + 'add' + '(' + "1" + ')'
print(function)

df.add(1)


In [132]:
eval('df.add(' + "1" + ')')

TypeError: can only concatenate str (not "int") to str